In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_non-reverse'
df = pd.read_pickle(f'{name_df}.pkl')

In [3]:
df

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_191,Col_192,Col_193,Col_194,Col_195,Col_196,Col_197,Col_198,Col_199,Col_200
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870510,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2870511,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,0
2870512,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,0,1
2870513,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,1,1,1,1,1


In [4]:
len(df)

2870515

In [5]:
new_columns = [f'Col_{i}' for i in range(1, 50 + 1)]
df_sort = df.sort_values(by = new_columns).reset_index(drop=True)

In [6]:
# Check the current data types of the DataFrame
print("Before Conversion:")
print(df_sort.dtypes)
print("\n")

for i in df_sort.columns:
    df_sort[i] = df_sort[i].astype(int)

# Check the data types after conversion
print("After Conversion:")
print(df_sort.dtypes)

Before Conversion:
Col_1      int64
Col_2      int64
Col_3      int64
Col_4      int64
Col_5      int64
           ...  
Col_196    int64
Col_197    int64
Col_198    int64
Col_199    int64
Col_200    int64
Length: 200, dtype: object


After Conversion:
Col_1      int64
Col_2      int64
Col_3      int64
Col_4      int64
Col_5      int64
           ...  
Col_196    int64
Col_197    int64
Col_198    int64
Col_199    int64
Col_200    int64
Length: 200, dtype: object


In [7]:
len(df_sort.columns)

200

In [8]:
# Step 1: Prepare Data
amount_features = len(df_sort.columns) - SIZE*SIZE #the previous boards
features = df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  2583463
len x test:  287052
len y train:  2583463
len y test:  287052
1017 14


,train,test
accuracy,1.0,1.0
precision,1.0,1.0
recall,1.0,1.0
f1,1.0,1.0


In [9]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\forward_model\\decision_tree_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.828627  0.827293
precision  0.603739  0.600747
recall     0.471763  0.470011
f1         0.529654  0.527398
63 5
              train      test
accuracy   0.835359  0.834375
precision  0.611485  0.609452
recall     0.534891  0.535070
f1         0.570629  0.569844
123 6
              train      test
accuracy   0.845666  0.844192
precision  0.638093  0.635022
recall     0.567041  0.564533
f1         0.600472  0.597706
229 7
              train      test
accuracy   0.874844  0.873295
precision  0.701121  0.698036
recall     0.676453  0.673259
f1         0.688566  0.685424
405 8
              train      test
accuracy   0.922675  0.921805
precision  0.800428  0.799654
recall     0.828517  0.825415
f1         0.814231  0.812331
661 9
              train      test
accuracy   0.973082  0.972399
precision  0.910327  0.908395
recall     0.963282  0.962432
f1         0.936056  0.934634
825 10
              train      test
accuracy   0.985553  0.985602

In [10]:
amount_features = len(df_sort.columns) - SIZE*SIZE #the previous boards
features = df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 20, 42)
    name_file = f"{PATH_MODELS}\\forward\\dt_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
1065 14
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
1
1017 15
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
2
969 12
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
3
1503 20
              train      test
accuracy   0.999889  0.999787
precision  0.999571  0.999371
recall     0.999888  0.999592
f1         0.999730  0.999481
4
1057 13
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
5
1149 14
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
6
1081 14
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
7
993 14
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       